In [1]:
# 1. Import knihoven
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv
import os

In [2]:
# 2. Vložení openAI API klíče ze skrytého souboru
try:
    from dotenv import load_dotenv
    load_dotenv()
except:
    pass

api_klic = os.getenv("OPENAI_API_KEY")

In [3]:
# 3. Předání API klíče a iniciace clienta
client = OpenAI(api_key=api_klic)

In [ ]:
# 4a. Načtení předfiltrovaných dat - ROHLÍK
rohlik_df = pd.read_csv('output/p_01_ingredience_rohlik_predfiltrovane.csv')

In [ ]:
# 4b.Načtení předfiltrovaných dat - KOŠÍK
kosik_df = pd.read_csv('output/p_01_ingredience_kosik_predfiltrovane.csv')

In [ ]:
# 5. Načtení dat o ingredincích a receptech
recept_ingred = pd.read_csv('data/recept_seznam_ingredienci.csv')

recepty_pro_ingredienci = (
    recept_ingred.groupby("ingredience_nazev")["recept_nazev"]
    .apply(lambda x: ", ".join(sorted(set(x))))
    .reset_index()
    .rename(columns={"recept_nazev": "seznam_receptu", "ingredience_nazev": "Ingredience"})
)

In [ ]:
# 6a. Sloučení s původní tabulkou ingrediencí - ROHLÍK
rohlik = rohlik_df.merge(recepty_pro_ingredienci, on="Ingredience", how="left")


In [ ]:
# 6b. Sloučení s původní tabulkou ingrediencí - KOŠÍK
kosik = kosik_df.merge(recepty_pro_ingredienci, on="Ingredience", how="left")


In [ ]:
# 7a. Ověřování vhodnosti produktů pro KOSIK

kosik['Vhodný produkt'] = None

for index, row in kosik.iterrows():
    surovina = row['Ingredience']
    produkt = row['Produkt']
    kategorie = row['Kategorie']
    recepty = row["seznam_receptu"]

    completion = client.chat.completions.create(
                    model="gpt-4o",
                    messages=[
                            {"role": "system",
                             "content": "Jsi nápomocný asistent pro hledání vhodných produktů pro vaření či pečení podle receptu. Buď striktní, např. 'vanilkové mléko' není vhodný produkt pro 'mléko' ani do sladkých receptů jako je bábovka. Zároveň ale 'TTD krupice cukr bílý' je vhodný produkt pro surovinu 'krupicový cukr'. Pro hovězí pečeni na svíčkovou omáčku se hodí např. hovězí zadní, karabáček, květová špička, loupaná nebo kulatá plec. Na guláš chceme kližku. Neposuzuj, zda daná surovina patří do daných receptů, např. pokud hledám skořici do guláše, najdi vhodnou skořici. Odpovídej True nebo False."},
                             {"role": "user",
                             "content": "Hledám vhodný produkt pro tuto surovinu: " + surovina + "pro recepty na tyto pokrmy: " + recepty + "Je tento produkt vhodný?" + produkt}
                            ]
                    )
    
    # zde vyextrahujete obsah odpovědi
    response_text = completion.choices[0].message.content

    kosik.at[index, 'Vhodný produkt'] = response_text

In [ ]:
# 8a. Export Kosik to csv
kosik.to_csv('output/p_02_ingredience_kosik_pouze_relevantni_produkty.csv', index=False)

In [ ]:
# 9a. Vyfiltrování pouze relevantních produktů a výběr jednoho nejlevnějšího (jednotková cena)

# Filter rows where 'Vhodný produkt' is True
kosik_final = kosik[kosik['Vhodný produkt'] == 'True']

kosik_final = kosik_final.sort_values(by=['Ingredience', 'Jednotková cena'], ascending=[True, True])

# Export this mid_step to csv
kosik_final.to_csv('output/p_03_ingredience_kosik_almost_final.csv', index=False)

# For each 'Ingredience', select the product with the lowest 'Jednotková cena'
kosik_final = kosik_final.loc[kosik_final.groupby('Ingredience')['Jednotková cena'].idxmin()]

# Fix Kosik images
kosik_final["IMG"] = kosik_final["IMG"] \
    .str.replace("WIDTH", "860") \
    .str.replace("HEIGHT", "800")

# Export Kosik to csv
kosik_final.to_csv('output/p_04_ingredience_kosik_final.csv', index=False)

In [ ]:
# 7b. Ověřování vhodnosti produktů pro ROHLIK

rohlik['Vhodný produkt'] = None

for index, row in rohlik.iterrows():
    surovina = row['Ingredience']
    produkt = row['Produkt']
    kategorie = row['Kategorie']
    recepty = row["seznam_receptu"]

    completion = client.chat.completions.create(
                    model="gpt-4o",
                    messages=[
                            {"role": "system",
                             "content": "Jsi nápomocný asistent pro hledání vhodných produktů pro vaření či pečení podle receptu. Buď striktní, např. 'vanilkové mléko' není vhodný produkt pro 'mléko' ani do sladkých receptů jako je bábovka. Zároveň ale 'TTD krupice cukr bílý' je vhodný produkt pro surovinu 'krupicový cukr'. Pro hovězí pečeni na svíčkovou omáčku se hodí např. hovězí zadní, karabáček, květová špička, loupaná nebo kulatá plec. Na guláš chceme kližku. Neposuzuj, zda daná surovina patří do daných receptů, např. pokud hledám skořici do guláše, najdi vhodnou skořici Odpovídej True nebo False."},
                             {"role": "user",
                             "content": "Hledám vhodný produkt pro tuto surovinu: " + surovina + "pro recepty na tyto pokrmy: " + recepty + "Je tento produkt vhodný?" + produkt}
                            ]
                    )
    
    # zde vyextrahujete obsah odpovědi
    response_text = completion.choices[0].message.content

    rohlik.at[index, 'Vhodný produkt'] = response_text

In [ ]:
# 8b. Export Rohlik to csv
rohlik.to_csv('output/p_02_ingredience_rohlik_pouze_relevantni_produkty.csv', index=False)


In [ ]:
# 9b. Vyfiltrování pouze relevantních produktů a výběr jednoho nejlevnějšího (jednotková cena)

# Filter rows where 'Vhodný produkt' is True
rohlik_final = rohlik[rohlik['Vhodný produkt'] == 'True']

rohlik_final = rohlik_final.sort_values(by=['Ingredience', 'Jednotková cena'], ascending=[True, True])

# Export this mid_step to csv
rohlik_final.to_csv('output/p_03_ingredience_rohlik_almost_final.csv', index=False)

# For each 'Ingredience', select the product with the lowest 'Jednotková cena'
rohlik_final = rohlik_final.loc[rohlik_final.groupby('Ingredience')['Jednotková cena'].idxmin()]

# Export Kosik to csv
rohlik_final.to_csv('output/p_04_ingredience_rohlik_final.csv', index=False)